# Publication post-processor

In [4]:
import pandas as pd

In [8]:
df = pd.read_parquet('../output/progress.parquet')
df.tail()

,dirname,title,authors,publisher,year,isbn,doi,llm_isbn,llm_doi,summary,cover_image
129,/Volumes/SJF/CVI_OPACID_SJF_9788089313013,CVI_OPACID_SJF_9788089313013,[],,,,,,,,
130,/Volumes/SJF/CVI_OPACID_SJF_9788089313358,CVI_OPACID_SJF_9788089313358,[],,,,,,,,
131,/Volumes/SJF/CVI_OPACID_SJF_ZB_uloh_1,CVI_OPACID_SJF_ZB_uloh_1,[],,,,,,,,
132,/Volumes/SJF/CVI_OPACID_SJF_ZB_uloh_3,CVI_OPACID_SJF_ZB_uloh_3,[],,,,,,,,
133,/Volumes/SJF/CVI_OPACID_SJF_ZB_uloh_4,CVI_OPACID_SJF_ZB_uloh_4,[],,,,,,,,


In [18]:
clean = df.dropna(subset=['authors'])